## Library

In [1]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


## Set Pipeline Variable

In [2]:
PIPELINE_NAME = "movie-reviews-pipeline"
SCHEMA_PIPELINE_NAME = "movie-reviews-tfdv-schema"

#Directory to store the artifacts that will be generated
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [3]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Loading

In [4]:
# read csv
pd.reset_option('^display.', silent=True)

DATA_PATH = 'data'
df = pd.read_csv(os.path.join(DATA_PATH, 'IMDB Dataset.csv'))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
#sentiment count
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
# label encoding
# 1 if 'sentiment'=='positive' else 0
temp=[]
for i in range(len(df)):
    if (df['sentiment'][i]=='positive'):
        temp.append(1)
    else:
        temp.append(0)
df['sentiment']=temp
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
# save data
ENCODE_CSV_PATH = 'data/encode_data'

# create new directory to save new csv
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
    if not os.path.exists(ENCODE_CSV_PATH):
        os.makedirs(ENCODE_CSV_PATH)
else:
    if not os.path.exists(ENCODE_CSV_PATH):
        os.makedirs(ENCODE_CSV_PATH)

# saving csv files
df.to_csv(os.path.join(ENCODE_CSV_PATH, 'movie_reviews_sentiment.csv'), index=False)

# assign new df
df = pd.read_csv(os.path.join(ENCODE_CSV_PATH, 'movie_reviews_sentiment.csv'))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
# define root
DATA_ROOT = ENCODE_CSV_PATH

## Data Ingestion

In [10]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [11]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

### summary statistics

In [12]:
# summary statistics
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)


interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Data Schema

In [14]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"]
)


interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [16]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',INT,required,,-


### Anomaly Detection

In [17]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [18]:
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

In [20]:
TRANSFORM_MODULE_FILE = "movie_reviews_transform.py"

In [23]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY = "sentiment"
FEATURE_KEY = "review"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.
    """

    outputs = {}

    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Overwriting movie_reviews_transform.py


In [24]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\movie-reviews-pipeline\Transform\transform_graph\7\.temp_path\tftransform_tmp\dc72f593ff60445982752951fd4e3d7a\assets


INFO:tensorflow:Assets written to: pipelines\movie-reviews-pipeline\Transform\transform_graph\7\.temp_path\tftransform_tmp\dc72f593ff60445982752951fd4e3d7a\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Development

### Hyperparameter Tuning

- [Forum Discussion Tuner from Dicoding](https://www.dicoding.com/academies/443/discussions/211900)
- [Tuner from Keras](https://keras.io/keras_tuner/)
- [Tuner TFX Pipeline Component from Tensorflow](https://www.tensorflow.org/tfx/guide/tuner?hl=id)

In [25]:
TUNER_MODULE_FILE = 'movie_reviews_tuner.py'

In [26]:
%%writefile {TUNER_MODULE_FILE}
import os
import keras_tuner
import tensorflow as tf
import tensorflow_transform as tft
from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

# set parameters
LABEL_KEY = "sentiment"
FEATURE_KEY = "review"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    embedding_dim=hp.Int('embedding_dim', min_value=16, max_value=128, step=16)
    lstm_units=hp.Int('lstm_units', min_value=16, max_value=128, step=16)
    num_layers=hp.Choice('num_layers', values=[1, 2, 3])
    dense_units=hp.Int('dense_units', min_value=16, max_value=128, step=16)
    dropout_rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

    reshaped_inputs = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_inputs)
    x = layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_layers):
        x = layers.Dense(dense_units, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    mode='max',
    verbose=1,
    patience=10
)

def tuner_fn(fn_args: FnArgs) -> None:
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files[0],  tf_transform_output, 10)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )

    # Build the model tuner
    tuner = keras_tuner.Hyperband(
        hypermodel=lambda hp: model_builder(hp),
        objective=keras_tuner.Objective('val_binary_accuracy', direction='max'),
        max_epochs=10,
        factor=3,
        directory=fn_args.working_dir,
        project_name='movie_reviews_keras_tuner'
    )

    return TunerFnResult(
        tuner      = tuner,
        fit_kwargs = {
            'callbacks': [early_stop_callback],
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps
        }
    )


Writing movie_reviews_tuner.py


In [27]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 review_xf (InputLayer)      [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 32)               4224      
 l)                                                              
                                                                 
 dense (Dense)               (None, 16)                528   

c:\Users\nurmuhimawann\Documents\GitHub\movie-reviews-classification\venv\lib\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


   4672/Unknown - 3182s 677ms/step - loss: 0.3134 - binary_accuracy: 0.8603

KeyboardInterrupt: 

### Training function

In [28]:
TRAINER_MODULE_FILE = "movie_reviews_trainer.py"

In [29]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "sentiment"
FEATURE_KEY = "review"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
            tf_transform_output,
            num_epochs,
            batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset


# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)


    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]

    )

    # print(model)
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)


    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])

    # Build the model
    model = model_builder()


    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing movie_reviews_trainer.py


### Trainer Component

In [30]:
from tfx.proto import trainer_pb2

trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 review_xf (InputLayer)      [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 64)                1088

INFO:tensorflow:Assets written to: pipelines\movie-reviews-pipeline\Trainer\model\9\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\movie-reviews-pipeline\Trainer\model\9\Format-Serving\assets


1000/1000 [==============================] - 102s 92ms/step - loss: 0.3747 - binary_accuracy: 0.8275 - val_loss: 0.3970 - val_binary_accuracy: 0.8332
Epoch 2/10
 999/1000 [============================>.] - ETA: 0s - loss: 0.2634 - binary_accuracy: 0.8801
Epoch 2: val_binary_accuracy did not improve from 0.83317
1000/1000 [==============================] - 73s 73ms/step - loss: 0.2633 - binary_accuracy: 0.8802 - val_loss: 0.5009 - val_binary_accuracy: 0.8184
Epoch 3/10
1000/1000 [==============================] - ETA: 0s - loss: 0.1995 - binary_accuracy: 0.9057
Epoch 3: val_binary_accuracy did not improve from 0.83317
1000/1000 [==============================] - 74s 74ms/step - loss: 0.1995 - binary_accuracy: 0.9057 - val_loss: 0.6637 - val_binary_accuracy: 0.8084
Epoch 4/10
1000/1000 [==============================] - ETA: 0s - loss: 0.1548 - binary_accuracy: 0.9296
Epoch 4: val_binary_accuracy did not improve from 0.83317
1000/1000 [==============================] - 74s 74ms/step - lo


Epoch 7: val_binary_accuracy did not improve from 0.83317
1000/1000 [==============================] - 41s 41ms/step - loss: 0.0612 - binary_accuracy: 0.9767 - val_loss: 1.1693 - val_binary_accuracy: 0.7970
INFO:tensorflow:struct2tensor is not available.


...

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


INFO:tensorflow:Assets written to: pipelines\movie-reviews-pipeline\Trainer\model\9\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\movie-reviews-pipeline\Trainer\model\9\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Model Analysis and Validation

### Resolver Component

In [31]:
# resolver
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluator Component

In [33]:
# evaluator configuration
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='sentiment')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}
                    ),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001}
                    )
                )
            )
        ])
    ]

)

In [34]:
# evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

### Result: interactive visualization

In [35]:
# visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Deployment Model

### Pusher Component

In [36]:
# pusher
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/movie-reviews-sentiment-model')
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## Export Model

In [ ]:
!zip -r pipelines.zip pipelines/
!zip -r serving_model_dir.zip serving_model_dir/
!pip freeze > requirements.txt

## TF-Serving

In [38]:
import requests
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/movie-reviews-sentiment-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1680333859'}]}
